In [3]:
import mysql.connector
import pandas as pd
import jaconv

def get_table_list(host, user, password, database):
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=3306
    )

    cursor = db.cursor()
    
    cursor.execute("SHOW TABLES")

    tables = cursor.fetchall()
    
    # テーブル名の一覧を取得
    tables_list = [table[0] for table in tables]

    return tables_list

# 使用例:
host = "localhost"
user = "user"
password = "password"
database = "cookpad-recipe-db"
print(get_table_list(host, user, password, database))


['base_kondate_likes', 'base_kondate_recipes', 'base_kondates', 'ingredients', 'kondate_categories', 'kondate_category_user_kondates', 'recipes', 'search_categories', 'search_category_recipes', 'steps', 'tsukurepos', 'user_kondate_infos']


In [18]:
import mysql.connector
import configparser

def get_config(filename):
    config = configparser.ConfigParser()
    config.read(filename)
    return config

def execute_sql(config, sql):
    db = mysql.connector.connect(
        host="localhost",
        user=config['DataBase']['DB_USER'],
        password=config['DataBase']['DB_PASS'],
        database=config['DataBase']['DB_NAME'],
        port=int(config['Data']['cookpad_db_port'])
    )

    
    df = pd.read_sql(sql, db)

    return df
# 使用例:
config = get_config("../config.ini")  # INIファイルのパスを適切に設定します

In [19]:
sql = "SELECT * FROM recipes limit 5"  # あなたのSQLクエリをここに入れます
execute_sql(config, sql)

/tmp/ipykernel_10178/2582603440.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,id,user_id,title,description,serving_for,advice,history,published_at
0,ad7d585b06850f8437ff5fb97d3c7a823ff21bb1,4db4672a8627d602c973547ef9fd5c4e1f12010b,豚の角煮,煮れば煮るほどやわらかくなります。しかも余分なあぶらもとれるので、ヘルシーかもしんない。時間...,４人分,,,1998-04-23
1,4afce5687dc173ad4fef943b686582a1cd06e264,0fe7862022a5d5711ee9464edcf20849f73e5725,スペシャルピーマンの肉詰め,スペシャルとは、にんじんとれんこんが入っているからなのです。\r\n４種類の野菜をつかってる...,２人分,なし,,1998-04-25
2,030833ed4e8dab3aa1e9d75edc1681efb368434f,28c343bd04f2867114878fb67eb5a43b26832c18,簡単チーズリゾット,キャンプなどアウトドア向けの料理です。\r\n簡単にできてかなりいける。ご飯をちゃんと炊くの...,２人分,ひもじさの賜物…かな。,,1998-05-06
3,12d462eae4db630ea5280380ebc99da3318080ae,046cacf243398debf505aeb45e73e145b1343b49,よもぎもち,春になると、野山によもぎが生えるよね。それを見て、母と共に｢懐かしいおばーちゃんの味」のよも...,よもぎもち(20個くらい）,母の記憶（よもぎを摘んで、あとは傍で見ていただけ。でもすごい!）,,1998-05-14
4,d05671227be64db32f383ba61af225c354b0167f,046cacf243398debf505aeb45e73e145b1343b49,口あたりのいい紅茶,紅茶は苦い！だから嫌いだ。と思っていた自分が好きになったきっかけの紅茶。\r\n紅茶のいろは...,一人分,,,1998-06-01


In [24]:
sql = "SELECT DISTINCT name FROM ingredients"
execute_sql(config, sql)

/tmp/ipykernel_10178/2582603440.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,name
0,None
1,\t
2,\tシソ
3,\n\nかぼちゃ（皮を除く）
4,\n\n海苔
...,...
1353179,ﾟ･*:.｡.飾り.｡.:*･゜
1353180,￥100の杏仁豆腐or杏仁プリンの素
1353181,￥100の瓶入りトマトソース
1353182,￥塩


In [25]:
sql = "SELECT name FROM ingredients"
execute_sql(config, sql)

/tmp/ipykernel_10178/2582603440.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,name
0,しょうが
1,ニンニク
2,ねぎ
3,豚肉
4,砂糖
...,...
12725001,*ラカンカット(砂糖)
12725002,*マヨネーズ
12725003,*豆腐(1パック)
12725004,水
